In [1]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px #if using plotly
import folium
import warnings
import uuid
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_columns', None)  # display all columns
pd.options.display.float_format = '{:20,.2f}'.format  # suppress scientific notation

### RQ #2) Which specific flares are the most impactful? 

### RQ #3) Which specific block grous are being most impacted? 


Exploratory question; no hypothesis testing

EJ index = (The Environmental Indicator Percentile for Block Group) x (Demographic Index for Block Group)
  
a) Determine flaring impact metric. 

Something like: buffer_population x EJ Index x flare volume  

b) Calculate impact per flare  

c) Visualize top_x flares by impact  


### Read in files

In [3]:
#ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_counties.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

In [4]:
#ca_bg = gp.read_file("data/ca_bg_clean.shp")  # CA block groups, cleaned
# CA block groups merged with EJscreen data
ca_bg_joined = gp.read_file("data/ca_bg_joined_clean.shp")

# update col name for correct area b/c it gets saved as a truncated version when saved as a shapefile out of the 
# main data cleaning notebook
ca_bg_joined.rename(columns={'shape_ar_1':'bg_area'}, inplace=True)  # old:new.
print(f'BGs found: {len(ca_bg_joined)}')

BGs found: 25590


In [5]:
# just CA flares
ca_flares = gp.read_file("data/ca_flares_clean.shp") 

print(f'Flares found: {len(ca_flares)}')

Flares found: 1170


In [6]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_counties = ca_counties.to_crs(meters_crs)
ca_flares = ca_flares.to_crs(meters_crs)
ca_bg_joined = ca_bg_joined.to_crs(meters_crs)

### Update ca_flares df

In [7]:
col_list = ['flare_id','flare_cate','cnty_name','year','bcm_values','BCM_avg','geometry']

subset = ca_flares[col_list].copy()

# assign the subsetted copy back to the original variable name to keep total # of df names small 
# throughout project
ca_flares = subset

In [8]:
ca_flares.sample(2)

,flare_id,flare_cate,cnty_name,year,bcm_values,BCM_avg,geometry
458,cf1a7423957987,Refinery,Los Angeles County,2017,NaN,0.00,POINT (165134.921 -469587.135)
620,45ab6e72920615,Upstream,Ventura County,2013,NaN,0.00,POINT (67294.971 -411219.351)


### Update ca_bg_joined df

In [9]:
# subset to only cols needed
subset = ca_bg_joined[['bg_id', 'cnty_name', 'ACSTOTPOP', 'MINORPOP',
                           'D_PM25_2', 'bg_area', 'geometry']].copy()

# assign the subsetted copy back to the original variable name
ca_bg_joined = subset

ca_bg_joined.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

## Create "baseline" df for use in further BG-specific and flare-specific analysis

In [10]:
# set geometry columns
ca_bg_joined = ca_bg_joined.set_geometry('bg_geom')

# and rename for clarity
ca_flares.rename(columns={'geometry':'flarepts_geom'}, inplace=True)  # old:new. Match col names for merging
ca_flares = ca_flares.set_geometry('flarepts_geom')

In [11]:
len(ca_bg_joined['bg_id'].unique())

25590

In [43]:
#buffer_size=100
#buffer_size=1000
buffer_size=2000

In [44]:
buffer_col = f"{buffer_size}_buff_geom"
ca_flares[buffer_col] = ca_flares['flarepts_geom'].buffer(distance=buffer_size)

ca_flares = ca_flares.set_geometry(buffer_col)

# subset of desired columns from flares_df
flares_sjoin_subset = ca_flares[[buffer_col, 'year', 'bcm_values','BCM_avg', 'flare_id', 'flare_cate']]

# Left join the flares_sjoin_subset cols to the bg_socialdf based on if any of the flares geometries intersect 
# with the bg geometries
intersect_baseline = gp.sjoin(ca_bg_joined, flares_sjoin_subset, how='left', predicate='intersects')

# Bring in buffer col that is not brought in by the sjoin() function for some unknown reason. Frustrating.
intersect_baseline = pd.merge(intersect_baseline, flares_sjoin_subset[['flare_id', buffer_col]], 
                             on='flare_id', how='left')

# set geom to the buffers and filter df down to just these BGs that have a flare_id associated with them. 
# There's no need to calculate any of the following for non-intersected BGs. 
intersect_baseline = intersect_baseline.set_geometry(buffer_col)
intersect_baseline = intersect_baseline[intersect_baseline.is_valid].copy()
print(f'{buffer_size}m BG-Buffer intersections: {len(intersect_baseline)}')

2000m BG-Buffer intersections: 50100


In [45]:
# Iterate through each bg_id and flare_id pairing and calculate 
# the proportion of the buffer that intersects with the BG geom, relative to the total buffer size.
# We need to know the proportion of each buffer in each bg geom in order to assign the percentage of
# flare volume to each bg in the case of incomplete overlap.

# First check for null or invalid geometries in buffer_2000m column
intersect_baseline['buffer_valid'] = intersect_baseline[buffer_col].is_valid

# Initialize intersect proportion "b_int_prop" and intersect area "b_int_area" columns with NaN values
intersect_baseline[f'{buffer_size}_buff_int_prop'] = np.nan
intersect_baseline[f'{buffer_size}_buff_int_area'] = np.nan

# Loop through each block group and flare pairing
for index, row in intersect_baseline.iterrows():
    # Check if buffer_2000m geometry is valid
    if row['buffer_valid']:
        # Calculate intersection area
        intersection = row[buffer_col].intersection(row['bg_geom'])  # prop of buffer in the bg geom
        intersection_area = intersection.area
        # Calculate buffer area
        buffer_area = row[buffer_col].area
        # Calculate intersection proportion
        buffer_int_prop = intersection_area / buffer_area
        # Update int_prop and int_area columns with calculated values
        intersect_baseline.at[index, f'{buffer_size}_buff_int_prop'] = buffer_int_prop
        intersect_baseline.at[index, f'{buffer_size}_buff_int_area'] = intersection_area
        intersect_baseline.at[index, f'{buffer_size}_buff_tot_area'] = buffer_area

    else:
        # Assign 0 to b_int_prop and b_int_area if buffer_2000m geometry is invalid
        intersect_baseline.at[index, f'{buffer_size}_buff_int_prop'] = 0
        intersect_baseline.at[index, f'{buffer_size}_buff_int_area'] = 0
        intersect_baseline.at[index, f'{buffer_size}_buff_tot_area'] = 0


# Drop buffer_valid column as it's no longer needed
intersect_baseline.drop('buffer_valid', axis=1, inplace=True)

In [46]:
# Now a similar analysis but this time we want to know the BG-buffer intersection proportion relative to 
# the total area of the block group, in order to use that proportion to assign counts of BG demographic
# variables to the intersection area. 

# Combine all flares into a single multipolygon unit
temp = ca_flares.unary_union
all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=ca_flares.crs) # convert back to geodf for processing

intersect_baseline = intersect_baseline.set_geometry('bg_geom')

# Make sure our geom column is set correctly to "bg_geom" rather than the flare buffer
if intersect_baseline.geometry.name == "bg_geom":
    int_temp = gp.overlay(intersect_baseline, all_buffers, how='intersection')  

    int_temp.rename(columns={'geometry':f'{buffer_size}_bg_int_geom'}, inplace=True)  # old:new. For clarity

    # Set geometry to the intersections
    int_temp = int_temp.set_geometry(f'{buffer_size}_bg_int_geom')  

    # Create new 'area' column for the areas of the intersections
    int_temp[f'{buffer_size}_bg_int_area'] = int_temp.area

    # Calculate the proportion of each block group intersection relative to total block broup area
    int_temp[f'{buffer_size}_bg_int_prop'] = int_temp[f'{buffer_size}_bg_int_area'] / int_temp['bg_area']

    # Merge the new area and props variables
    intersect_baseline = pd.merge(intersect_baseline, 
                                 int_temp[['bg_id',
                                           f'{buffer_size}_bg_int_prop',
                                           f'{buffer_size}_bg_int_area', 
                                           f'{buffer_size}_bg_int_geom']], 
                                 on=['bg_id'], 
                                 how='left').fillna({f'{buffer_size}_bg_int_prop':0, 
                                                     f'{buffer_size}_bg_int_area':0})

else:
    print("Error: Geometry column is not named 'bg_geom'")

In [47]:
# Apply the proportion to each demographic variable to find counts by variable
# i.e. this answers "how many of x live in the X buffer radius intersection area?"
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    intersect_baseline[f'{buffer_size}_{var}_bg_int_count'] = (intersect_baseline[var] *
                                                               intersect_baseline[f'{buffer_size}_bg_int_prop'])

# Find *overall* proportions for each demo var by dividing var count by respective total population
# i.e. this answers "what overall percentage of this block group is minority status?"
intersect_baseline['MINORPOP_bg_totprop'] = intersect_baseline['MINORPOP'] / intersect_baseline['ACSTOTPOP']

# calculate percentage of minorities in intersection for each BG
intersect_baseline[f'{buffer_size}_bg_int_mnr_prop'] = (intersect_baseline[f'{buffer_size}_MINORPOP_bg_int_count']
                                                        / intersect_baseline[f'{buffer_size}_ACSTOTPOP_bg_int_count'])

In [48]:
# There are a ton of duplicate rows getting introduced at some point of the process. Ideally I'll come back and 
# troubleshoot, but in the meantime I'm just removing these dupes. 

df_baseline_all = intersect_baseline.drop_duplicates(subset=['bg_id', 'flare_id', 'year']).copy()

# I think the sjoin() functions ocassionally do weird things to data types. 
# Changing year back to an integer
df_baseline_all['year'] = df_baseline_all['year'].astype('int64')


In [49]:
len(df_baseline_all)

5010

In [50]:
# grab two specific bgs to check
df_baseline_all[df_baseline_all['bg_id'].isin([60379800331, 60379800111])]


,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,year,bcm_values,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop
720000,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",815.00,2014,NaN,0.00,1c636353194312,Refinery,"POLYGON ((167197.511 -469609.028, 167187.881 -...",0.11,"1,376,354.57","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
725000,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...","1,166.00",2018,0.00,0.00,1c636353194312,Refinery,"POLYGON ((167197.511 -469609.028, 167187.881 -...",0.11,"1,376,354.57","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
730000,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",581.00,2012,NaN,0.00,1c636353194312,Refinery,"POLYGON ((167197.511 -469609.028, 167187.881 -...",0.11,"1,376,354.57","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
735000,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...","1,049.00",2016,NaN,0.00,1c636353194312,Refinery,"POLYGON ((167197.511 -469609.028, 167187.881 -...",0.11,"1,376,354.57","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
740000,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",230.00,2020,NaN,0.00,1c636353194312,Refinery,"POLYGON ((167197.511 -469609.028, 167187.881 -...",0.11,"1,376,354.57","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965000,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",925.00,2015,NaN,0.00,da91f7f1682313,Refinery,"POLYGON ((166954.446 -469593.965, 166944.816 -...",0.54,"6,776,933.35","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00
4970000,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",691.00,2013,NaN,0.00,da91f7f1682313,Refinery,"POLYGON ((166954.446 -469593.965, 166944.816 -...",0.54,"6,776,933.35","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00
4975000,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",106.00,2019,NaN,0.00,da91f7f1682313,Refinery,"POLYGON ((166954.446 -469593.965, 166944.816 -...",0.54,"6,776,933.35","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00
4980000,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...","1,159.00",2018,NaN,0.00,da91f7f1682313,Refinery,"POLYGON ((166954.446 -469593.965, 166944.816 -...",0.54,"6,776,933.35","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00


# Aggregate data by BG and export files for Tableau

take baseline dfs and create copies clearly defined for this BG analysis, separate from the work that is needed
for the flare buffer analysis below. Both analyses modify these same two dfs in different ways and I want 
to make sure I keep the calculations separated

#### Standardize variables

EJ index and total population in the intersection(s) are already encoded at the level of a BG, so they're ready to normalize. 

Flare volume needs to multiplied by the proportion of the flare's buffer zone that resides in a BG, and then all (in the case of multiple flares in a BG) adjusted flare volumes summed up to generate a variable for total flare volume a BG is exposed to.  

In [ ]:
def bg_calc_adj_flare_vol(df, buffer_size):
    """
    Function for calculating total flare volume to be assigned to each BG, based on amount of overlap.
    
    "bcm_adj" = adjusted bcm value for a given flare_id and bg_id, based on the proportion of the buffer zone that 
    overlaps with a given bg_id. Equal to the bcm_avg * b_int_prop (the overlap)
    
    "bcm_adj_tot" = the sum of the bcm_adj values for a given bg_id
    """
    
    # calculate the product of bcm_avg and buff_int_prop for each flare and bg combination
    df[f'{buffer_size}_bcm_int_adj'] = df['BCM_avg'] * df[f'{buffer_size}_buff_int_prop']
    
    # sum the adjusted volumes for all flare and bg combinations
    adj_vol = df.groupby(['flare_id', 'bg_id'])[f'{buffer_size}_bcm_int_adj'].sum().reset_index()
    
    # sum those adjusted volumes for all flares in each block group
    bg_vol = adj_vol.groupby('bg_id')[f'{buffer_size}_bcm_int_adj'].sum().reset_index()
    
    # merge the total adjusted block group volumes with the original dataframe
    df = pd.merge(df, bg_vol, on='bg_id', how='left', suffixes=('', '_tot'))
    
    return df

In [ ]:
df_bg_intersect = df_baseline_all.groupby('bg_id').apply(bg_calc_adj_flare_vol, buffer_size=buffer_size)

In [ ]:
# Define the variables to standardize

#'bcm_adj_tot' = Sum of the adjusted flare volume(s) that intersect w the BG. 
#'D_PM25_2' = EJ index for the BG
#'ACSTOTPOP_int_count' = BG population that resides in the intersection. = BGtotpop * intersection_proportion

varlist = [f'{buffer_size}_bcm_int_adj_tot', 'D_PM25_2', f'{buffer_size}_ACSTOTPOP_bg_int_count']

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(df_bg_intersect[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

intersect_norm = intersect_norm.reset_index(drop=True)
df_bg_intersect = df_bg_intersect.reset_index(drop=True)

# Concatenate the new variables with the original DataFrame
df_bg_intersect = pd.concat([df_bg_intersect, intersect_norm], axis=1)

In [ ]:
# Create calculations of # of counties affected and counts of BGs according to different
# "bins" of overlap

df_bg_intersect[f'{buffer_size}_bg_int_25%orless'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= .25).sum()).sum())

df_bg_intersect[f'{buffer_size}_bg_int_50%orless'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= .5).sum()).sum())

df_bg_intersect[f'{buffer_size}_bg_int_75%orless'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= .75).sum()).sum())

df_bg_intersect[f'{buffer_size}_bg_int_anyint'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= 1).sum()).sum())



In [ ]:
# counties affected
temp = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', f'{buffer_size}_bg_int_geom']], 
                                         geometry=f'{buffer_size}_bg_int_geom')

counties_intersect = gp.sjoin(ca_counties, temp, how='inner', predicate='contains')  

df_bg_intersect[f'{buffer_size}_num_counties'] = len(counties_intersect['cnty_name'].unique())

In [ ]:
df_bg_intersect.sample(10)

In [ ]:
# There are a ton of duplicate rows getting introduced at some point of the process. Ideally I'll come back and 
# troubleshoot, but in the meantime I'm just removing these dupes. 
df_bg_intersect = df_bg_intersect.drop_duplicates(subset=['bg_id', 'flare_id'])

In [ ]:
len(df_bg_intersect)

In [ ]:
# # check calculations against two BGs
# # Should see a single "bcm_adj_tot" value per BG, and different adjusted "bcm_adj" values per flare_id 
# #depending on the amount of overlap.

# # filter the dataframe and format specified columns to display 4 decimal places
# filtered_df = df_bg_intersect_all[df_bg_intersect_all['bg_id'].isin([60379800331, 60379800111])].copy()
# filtered_df[['BCM_avg', f'{buffer_size}_bcm_int_adj', f'{buffer_size}_bcm_int_adj_tot', 
#              f'{buffer_size}_bcm_int_adj_tot_n']] = filtered_df[['BCM_avg', f'{buffer_size}_bcm_int_adj',
#                                                                  f'{buffer_size}_bcm_int_adj_tot', 
#                                                                  f'{buffer_size}_bcm_int_adj_tot_n']].apply(lambda x: x.apply(lambda y: '{:.4f}'.format(y)))


In [ ]:
# filtered_df

### Split geom data from rest of data b/c column names must be <10 characters for shapefiles and that makes naming my different columns unnecessarily confusing. I export a csv of everything but the geometry, export a shapefile of just the geometry, then join the two back together in Tableau. 

In [ ]:
df_bg_intersect.sample(1)

### 100

In [ ]:
df_bgintersect_geo100 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '100_bg_int_geom']], 
                                         geometry='100_bg_int_geom')

print(len(df_bgintersect_geo100[df_bgintersect_geo100.is_valid]))

df_bgintersect_100 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', f'{buffer_size}_bg_int_geom'], axis=1)

In [ ]:
# Add in a buffer_size col for filtering in Tableau
df_bgintersect_geo100['buffer_size'] = 100

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
df_bgintersect_geo100.rename(columns={'100_bg_int_geom':'bg_int_geom'}, inplace=True)  # old:new. Match col names for merging

In [ ]:
df_bgintersect_100.sample(2)

In [ ]:
df_bgintersect_geo100.sample(2)

In [ ]:
#df_bgintersect_geo100.to_file('data/df_bg_geo2000.shp', driver='ESRI Shapefile')

### 1000

In [ ]:
df_bgintersect_geo1000 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '1000_bg_int_geom']], 
                                         geometry='1000_bg_int_geom')

print(len(df_bgintersect_geo1000[df_bgintersect_geo1000.is_valid]))

df_bgintersect_1000 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', 
                                            f'{buffer_size}_bg_int_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
df_bgintersect_geo1000['buffer_size'] = 1000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
df_bgintersect_geo1000.rename(columns={'1000_bg_int_geom':'bg_int_geom'}, inplace=True)  # old:new.

In [ ]:
df_bgintersect_1000.head(2)

In [ ]:
#df_bgintersect_geo2000.to_file('data/df_bg_geo2000.shp', driver='ESRI Shapefile')

### 2000

In [ ]:
# Create a new geodataframe with just the id and geom columns
df_bgintersect_geo2000 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '2000_bg_int_geom']], 
                                         geometry='2000_bg_int_geom')

print(len(df_bgintersect_geo2000[df_bgintersect_geo2000.is_valid]))


# Subset the original dataframe and drop unnecessary columns
df_bgintersect_2000 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', 
                                         f'{buffer_size}_bg_int_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
df_bgintersect_geo2000['buffer_size'] = 2000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
df_bgintersect_geo2000.rename(columns={'2000_bg_int_geom':'bg_int_geom'}, inplace=True)  # old:new.

In [ ]:
df_bgintersect_2000.head(2)

In [ ]:
#df_bgintersect_geo5000.to_file('data/df_bg_geo5000.shp', driver='ESRI Shapefile')

### Combine

#### Social data

In [ ]:
def filter_dataframe(df, prefix):
    cols_to_keep = ['bg_id', 'flare_id'] + [col for col in df.columns if col.startswith(prefix)]
    return df[cols_to_keep].copy()

In [ ]:
# filter the 2nd and 3rd dataframes to only include columns starting with the buffer cols
# eg '5000_' plus the ID columns
# all the non "5000_" columns in these dfs are identical to the first df, so can be ignored.
# select columns that start with the appropriate prefix and keep 'bg_id' and 'flare_id'

df_bgintersect_100_filtered = filter_dataframe(df_bgintersect_100, '100_')
df_bgintersect_1000_filtered = filter_dataframe(df_bgintersect_1000, '1000_')

In [ ]:
df_bg_intersect_temp = df_bgintersect_2000.merge(df_bgintersect_1000_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')


df_bg_intersect_all = df_bg_intersect_temp.merge(df_bgintersect_100_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')

In [ ]:
len(df_bg_intersect_all)

In [ ]:
df_bg_intersect_all[df_bg_intersect_all['bg_id'].isin([60372911201])]


In [ ]:
len(df_bg_intersect_all)

In [ ]:
df_bg_intersect_all.to_csv('data/df_bg_impact_all.tsv', sep='\t', index=False)

#### Geo data

In [ ]:
df_bg_intersect_geoall = pd.concat([df_bgintersect_geo100, 
                                    df_bgintersect_geo1000,
                                    df_bgintersect_geo2000])

In [ ]:
501-216-118

In [ ]:
# num intersections at 2000m: 501
# at 1000m: 216
# at 100m: 118
# Should equal 835 total
len(df_bg_intersect_geoall)

In [ ]:
df_bg_intersect_geoall.sample(10)

In [ ]:
df_bg_intersect_geoall = df_bg_intersect_geoall.set_geometry('bg_int_geom')
df_bg_intersect_geoall.to_file('data/df_bg_geoall.shp', driver='ESRI Shapefile')

# Aggregate by Flare ID and export files for Tableau

Take baseline dfs and create copies clearly defined for this flare buffer analysis, 
separate from the work that is needed for the BG buffer analysis above. 
Both analyses modify these same two dfs in different ways; need to ensure I keep the calculations separated.
Plus, they'll go into Tableau as different dataframes anyway. 

In [51]:
# Use the baseline df as the starting point

df_baseline_all.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,year,bcm_values,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop
5942000,60710125001,San Bernardino County,1286,1218,86.43,"1,820,607.61","POLYGON ((246851.603 -437537.569, 246854.512 -...",349.00,2021,NaN,0.00,99b1dbeb266912,Upstream,"POLYGON ((247587.514 -437618.082, 247577.883 -...",0.03,"436,090.14","12,546,193.96",0.24,"436,090.14","POLYGON ((246851.603 -437537.569, 246854.512 -...",308.04,291.75,0.95,0.95


In [52]:
# Define the variables to aggregate

# BCM_avg: No adjustments needed because bcm is already attached to the flare unit of analysis
# D_PM25_D: Create new var of avg PM25 score for all BGs in each buffer 
# ACSTOTPOP_bg_int_count: Create new var of total population in all intersections in each buffer

agg_dict = {'D_PM25_2': 'mean', f'{buffer_size}_ACSTOTPOP_bg_int_count': 'sum', f'{buffer_size}_MINORPOP_bg_int_count': 'sum'}

# set up df for flare-specific calcs
flare_intersect = df_baseline_all.copy()

# Group the block groups by flare ID and aggregrate the selected vars
flare_agg = df_baseline_all.groupby('flare_id').agg(agg_dict)

# Rename the output variables
flare_agg = flare_agg.rename(columns={'D_PM25_2': f'{buffer_size}_int_pm25_avg', 
                                      f'{buffer_size}_ACSTOTPOP_bg_int_count': f'{buffer_size}_ACSTOTPOP_bg_int_sum',
                                     f'{buffer_size}_MINORPOP_bg_int_count': f'{buffer_size}_MINORPOP_bg_int_sum'})

# Join the aggregated variables with the original columns
flare_intersect = flare_intersect.merge(flare_agg, on='flare_id')

# add a new column for proportion of minorities within the given buffer zone
flare_intersect[f'{buffer_size}_flare_minor_prop'] = (flare_intersect[f'{buffer_size}_MINORPOP_bg_int_sum'] / 
                                                      flare_intersect[f'{buffer_size}_ACSTOTPOP_bg_int_sum'])


In [53]:
flare_intersect.sample(3)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,year,bcm_values,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop
2868,61110096002,Ventura County,2906,707,9.92,"24,853,672.44","POLYGON ((67738.131 -414975.718, 67833.685 -41...","1,088.00",2018,0.00,0.00,45ab6e72920615,Upstream,"POLYGON ((69294.971 -411219.351, 69285.340 -41...",0.05,"596,719.84","12,546,193.96",0.04,"1,023,511.56","POLYGON ((68253.538 -413085.630, 68251.998 -41...",119.67,29.12,0.24,0.24,19.62,"24,051.45","13,102.20",0.54
4289,60133141062,Contra Costa County,3385,3203,79.17,"447,675.15","POLYGON ((-169919.509 2071.581, -169903.823 20...",324.00,2021,0.01,0.01,db6f84a8406417,Upstream,"POLYGON ((-167598.970 344.705, -167608.600 148...",0.01,"171,376.10","12,546,193.96",0.38,"171,376.10","POLYGON ((-169919.509 2071.581, -169903.823 20...","1,295.82","1,226.15",0.95,0.95,51.24,"121,509.02","103,805.46",0.85
4028,61110029051,Ventura County,2893,2132,29.21,"20,363,175.76","POLYGON ((67526.239 -419885.068, 67663.390 -41...",976.00,2016,NaN,0.00,29b67b94215591,Upstream,"POLYGON ((71737.988 -420210.310, 71728.358 -42...",0.63,"7,869,336.28","12,546,193.96",0.39,"7,869,336.28","POLYGON ((67771.399 -419853.961, 67819.541 -41...","1,118.00",823.91,0.74,0.74,23.60,"17,503.94","10,831.67",0.62


In [54]:
# Define the variables to standardize

varlist = ['BCM_avg', f'{buffer_size}_int_pm25_avg', f'{buffer_size}_ACSTOTPOP_bg_int_sum']  

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(flare_intersect[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

# Concatenate the new variables with the original DataFrame
flare_intersect = pd.concat([flare_intersect, intersect_norm], axis=1)

In [55]:
flare_intersect.sample(3)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,year,bcm_values,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop,BCM_avg_n,2000_int_pm25_avg_n,2000_ACSTOTPOP_bg_int_sum_n
4312,60133552022,Contra Costa County,2148,1870,50.32,"19,561,543.93","POLYGON ((-177596.793 396.875, -177578.032 425...",675.00,2013,NaN,0.01,db6f84a8406417,Upstream,"POLYGON ((-167598.970 344.705, -167608.600 148...",0.02,"227,657.03","12,546,193.96",0.01,"227,657.03","POLYGON ((-171383.760 -553.799, -171356.298 -5...",25.00,21.76,0.87,0.87,51.24,"121,509.02","103,805.46",0.85,2.19,0.31,0.18
4144,60133132051,Contra Costa County,1839,1545,48.71,"1,481,529.02","POLYGON ((-169916.595 2003.918, -169915.487 20...","1,142.00",2018,NaN,0.01,54040d4c310483,Upstream,"POLYGON ((-167588.809 339.717, -167598.440 143...",0.04,"472,295.69","12,546,193.96",0.32,"472,295.69","MULTIPOLYGON (((-167741.544 1106.127, -167815....",586.25,492.53,0.84,0.84,51.24,"121,509.02","103,805.46",0.85,2.75,0.31,0.18
4296,60133141052,Contra Costa County,3516,3317,59.23,"616,116.21","POLYGON ((-171047.294 2917.300, -171030.554 29...",792.00,2014,NaN,0.01,db6f84a8406417,Upstream,"POLYGON ((-167598.970 344.705, -167608.600 148...",0.00,"5,881.86","12,546,193.96",0.01,"5,881.86","POLYGON ((-170363.356 2192.815, -170363.386 21...",33.57,31.67,0.94,0.94,51.24,"121,509.02","103,805.46",0.85,2.19,0.31,0.18


### 100

In [26]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flareintersect_geo100 = gp.GeoDataFrame(flare_intersect[['flare_id', '100_buff_geom']].copy(), 
                                      geometry='100_buff_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flareintersect_100 = flare_intersect.drop(['bg_geom', '100_bg_int_geom', '100_buff_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
flareintersect_geo100['buffer_size'] = 100

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
flareintersect_geo100.rename(columns={'100_buff_geom':'buff_geom'}, inplace=True)  # old:new.

In [27]:
flareintersect_100.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,year,bcm_values,BCM_avg,flare_id,flare_cate,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,MINORPOP_bg_totprop,100_bg_int_mnr_prop,100_int_pm25_avg,100_ACSTOTPOP_bg_int_sum,100_MINORPOP_bg_int_sum,100_flare_minor_prop,BCM_avg_n,100_int_pm25_avg_n,100_ACSTOTPOP_bg_int_sum_n
593,61110012041,Ventura County,2831,1652,29.14,"120,975,492.62",591.00,2013,NaN,0.00,045e6f25140159,Upstream,1.00,"31,365.48","31,365.48",0.00,"83,592.84",1.96,1.14,0.58,0.58,29.14,19.56,11.42,0.58,0.60,-0.32,-0.04


### 1000

In [41]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flareintersect_geo1000 = gp.GeoDataFrame(flare_intersect[['flare_id', '1000_buff_geom']].copy(), 
                                      geometry='1000_buff_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flareintersect_1000 = flare_intersect.drop(['bg_geom', '1000_bg_int_geom', '1000_buff_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
flareintersect_geo1000['buffer_size'] = 1000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
flareintersect_geo1000.rename(columns={'1000_buff_geom':'buff_geom'}, inplace=True)  # old:new.

In [42]:
flareintersect_1000.sample(20)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,year,bcm_values,BCM_avg,flare_id,flare_cate,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,1000_bg_int_mnr_prop,1000_int_pm25_avg,1000_ACSTOTPOP_bg_int_sum,1000_MINORPOP_bg_int_sum,1000_flare_minor_prop,BCM_avg_n,1000_int_pm25_avg_n,1000_ACSTOTPOP_bg_int_sum_n
1434,60133132032,Contra Costa County,2637,2125,47.88,"2,606,490.18",207.00,2020,NaN,0.01,db6f84a8406417,Upstream,0.13,"422,191.34","3,136,548.49",0.16,"422,883.57",427.83,344.77,0.81,0.81,42.33,"9,512.33","7,389.75",0.78,2.16,0.09,0.16
652,60290033043,Kern County,914,800,65.94,"502,699,881.91",410.00,2017,NaN,0.00,3af2619e757995,Upstream,1.00,"3,136,548.49","3,136,548.49",0.03,"12,594,194.50",22.90,20.04,0.88,0.88,65.94,228.99,200.42,0.88,-0.21,1.12,-0.61
2145,61110097003,Ventura County,733,62,3.13,"2,383,596,963.76",840.00,2015,NaN,0.00,9889e389860660,Upstream,1.00,"3,136,548.49","3,136,548.49",0.00,"3,389,222.26",1.04,0.09,0.08,0.08,3.13,10.42,0.88,0.08,-0.15,-1.63,-0.62
1211,61110012041,Ventura County,2831,1652,29.14,"120,975,492.62",120.00,2020,NaN,0.00,4567aed4989022,Upstream,0.91,"2,867,572.95","3,136,548.49",0.03,"3,101,283.84",72.57,42.35,0.58,0.58,26.46,"4,377.14","2,577.28",0.59,-0.04,-0.61,-0.26
1218,61110012041,Ventura County,2831,1652,29.14,"120,975,492.62","1,056.00",2018,NaN,0.00,4567aed4989022,Upstream,0.91,"2,867,572.95","3,136,548.49",0.03,"3,101,283.84",72.57,42.35,0.58,0.58,26.46,"4,377.14","2,577.28",0.59,-0.04,-0.61,-0.26
1144,60310017031,Kings County,3013,2770,66.83,"770,201,466.11",773.00,2014,NaN,0.00,a747c536488467,Upstream,1.00,"3,136,548.49","3,136,548.49",0.00,"3,603,992.57",14.10,12.96,0.92,0.92,66.83,140.99,129.62,0.92,-0.39,1.16,-0.61
1948,60190079032,Fresno County,4735,3754,28.51,"1,436,574,980.80",666.00,2013,0.00,0.00,5085898a781258,Upstream,0.37,"1,157,684.73","3,136,548.49",0.00,"1,157,684.73",3.82,3.03,0.79,0.79,56.86,147.03,137.64,0.94,-0.53,0.72,-0.61
1104,60310017031,Kings County,3013,2770,66.83,"770,201,466.11",658.00,2013,0.00,0.00,75a75212412802,Upstream,1.00,"3,136,548.49","3,136,548.49",0.00,"3,603,992.57",14.10,12.96,0.92,0.92,66.83,140.99,129.62,0.92,-0.27,1.16,-0.61
104,60379800141,Los Angeles County,45,41,81.28,"7,451,821.32",107.00,2019,NaN,0.00,cf1a7423957987,Refinery,0.04,"116,159.54","3,136,548.49",0.09,"703,531.99",4.25,3.87,0.91,0.91,84.81,147.97,131.17,0.89,-0.08,1.94,-0.61
1630,61110097001,Ventura County,1125,1028,16.36,"147,532,134.93","1,063.00",2018,NaN,0.00,58373432792284,Upstream,0.97,"3,033,285.12","3,136,548.49",0.02,"3,328,806.71",25.38,23.20,0.91,0.91,17.23,287.43,245.97,0.86,-0.21,-1.01,-0.60


### 2000

In [56]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flareintersect_geo2000 = gp.GeoDataFrame(flare_intersect[['flare_id', '2000_buff_geom']].copy(), 
                                      geometry='2000_buff_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flareintersect_2000 = flare_intersect.drop(['bg_geom', '2000_bg_int_geom', '2000_buff_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
flareintersect_geo2000['buffer_size'] = 2000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
flareintersect_geo2000.rename(columns={'2000_buff_geom':'buff_geom'}, inplace=True)  # old:new.

In [57]:
flareintersect_2000.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,year,bcm_values,BCM_avg,flare_id,flare_cate,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop,BCM_avg_n,2000_int_pm25_avg_n,2000_ACSTOTPOP_bg_int_sum_n
203,60376502004,Los Angeles County,1790,1305,38.69,"1,131,796.81",110.00,2019,NaN,0.00,7d5801fd595483,Refinery,0.09,"1,131,796.81","12,546,193.96",1.00,"1,131,796.81","1,790.00","1,305.00",0.73,0.73,40.59,"155,665.99","106,228.68",0.68,-0.52,-0.22,0.52


### Combine

#### Geo data

In [58]:
flareintersect_geo100.sample(1)

,flare_id,buff_geom,buffer_size
671,fc31a7e3701977,"POLYGON ((47097.528 -304252.983, 47097.046 -30...",100


In [59]:
flareintersect_geo1000.sample(1)

,flare_id,buff_geom,buffer_size
843,41589855183790,"POLYGON ((-10949.702 -150361.388, -10954.517 -...",1000


In [60]:
flareintersect_geo2000.sample(1)

,flare_id,buff_geom,buffer_size
2394,99b1dbeb266912,"POLYGON ((247587.514 -437618.082, 247577.883 -...",2000


In [61]:
df_flareintersect_geoall = pd.concat([flareintersect_geo100, 
                                    flareintersect_geo1000,
                                    flareintersect_geo2000])

In [62]:
df_flareintersect_geoall.sample(10)

,flare_id,buff_geom,buffer_size
89,7d5801fd595483,"POLYGON ((156983.129 -460981.588, 156973.498 -...",2000
712,eacf54cb909186,"POLYGON ((32563.729 -285943.545, 32563.248 -28...",100
4516,c9f7e0b7926195,"POLYGON ((268033.062 -447165.112, 268023.431 -...",2000
793,f6c36958663369,"POLYGON ((95120.548 -332103.299, 95115.733 -33...",1000
1814,1eb6c54b458336,"POLYGON ((203158.098 -439404.603, 203148.468 -...",2000
369,75a75212412802,"POLYGON ((4190.624 -235183.764, 4190.142 -2351...",100
2588,a36a5fe6112227,"POLYGON ((69347.393 -411227.380, 69337.763 -41...",2000
3692,d4760e0d180348,"POLYGON ((95958.963 -331902.632, 95949.332 -33...",2000
2804,ca3c8041420679,"POLYGON ((69346.315 -411230.717, 69336.684 -41...",2000
3212,ba6266cf427683,"POLYGON ((-182030.333 3537.499, -182039.963 33...",2000


In [63]:
df_flareintersect_geoall = df_flareintersect_geoall.set_geometry('buff_geom')
df_flareintersect_geoall.to_file('data/df_flare_geoall.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


#### Social data

In [68]:
# Keep the two relevant ID columns plus the prefix for the buffer-specific columns for each buffer size

def filter_dataframe(df, prefix):
    cols_to_keep = ['bg_id', 'flare_id', 'year'] + [col for col in df.columns if col.startswith(prefix)]
    return df[cols_to_keep].copy()

In [69]:
# filter the 2nd and 3rd dataframes to only include columns starting with the buffer cols
# eg '5000_' plus the ID columns
# all the non "5000_" columns in these dfs are identical to the first df, so can be ignored.
# select columns that start with the appropriate prefix and keep 'bg_id' and 'flare_id'

flareintersect_100_filtered = filter_dataframe(flareintersect_100, '100_')
flareintersect_1000_filtered = filter_dataframe(flareintersect_1000, '1000_')

In [70]:
df_flare_intersect_temp = flareintersect_2000.merge(flareintersect_1000_filtered, 
                                               on=['bg_id', 'flare_id' ,'year'], how='outer')


df_flare_intersect_all = df_flare_intersect_temp.merge(flareintersect_100_filtered, 
                                               on=['bg_id', 'flare_id', 'year'], how='outer')

In [71]:
df_flare_intersect_all.to_csv('data/df_flare_impact_all.tsv', sep='\t', index=False)

## Folium Mapping

In [ ]:
# for_map = df_final[['flare_id', 'BCM_avg', 'D_PM25_2', 'ACSTOTPOP_intersect_count', 'impact_metric', 'buffer_2000m']]
# for_map = gp.GeoDataFrame(for_map, geometry='buffer_2000m', crs=meters_crs)

# for_map.to_file("data/df_impactmetric_shp.shp", driver='ESRI Shapefile')

In [ ]:

# # Define the color scale and number of bins
# #color_scale = 'Reds'
# #num_bins = 10

# # Create a map centered on the first flare
# # Create a folium map with a center location
# m = folium.Map(location=[38.377158,-121.645792], zoom_start=6, tiles=None,overlay=False)  #start w lat/long roughly in center of CA
# base_map = folium.FeatureGroup(name='Base map', overlay=True, control=False)
# folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
# base_map.add_to(m)




In [ ]:
# # Feature Group: Buffers

# def style_function3(feature):
#     return {
#         'fillColor': 'red',
#         'color': 'red',
#         'fillOpacity': 0.05
#     }

# inter_all_bg = folium.FeatureGroup(name='Intersect BG', overlay=True)
# folium.GeoJson(
#     data=test["buffer_2000m"],
#     style_function=style_function3
# ).add_to(inter_all_bg)
# inter_all_bg.add_to(m)

In [ ]:
# # Feature Group: block group

# def style_function2(feature):
#     return {
#         'fillColor': 'blue',
#         'color': 'blue',
#         'fillOpacity': 0.05
#     }

# bg_bound = folium.FeatureGroup(name='Intersect BG', overlay=True)
# folium.GeoJson(
#     data=test["bg_geom"],
#     style_function=style_function2
# ).add_to(bg_bound)
# bg_bound.add_to(m)

In [ ]:
# style_function = lambda x: {'fillColor': '#ffffff', 
#                             'color':'#000000', 
#                             'fillOpacity': 0.1, 
#                             'weight': 0.1}
# highlight_function = lambda x: {'fillColor': '#999999', 
#                                 'color':'#999999', 
#                                 'fillOpacity': 0.50, 
#                                 'weight': 0.1}
# NIL = folium.features.GeoJson(
#     data = for_map,
#     style_function=style_function, 
#     control=False,
#     highlight_function=highlight_function, 
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['flare_id', 'BCM_avg_norm', 'D_PM25_2_norm', 
#              'ACSTOTPOP_intersect_count_norm', 'impact_metric'],# 'D_PM25_2', 'ACSTOTPOP', 'MINORPOP','shape_area_new', 'intersect_prop', 'intersect_area', 'MINORPOP_bg_totprop'],
#         style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#     )
# )

In [ ]:
# # add hover functionality as child to map, add layering, display map
# m.add_child(NIL)
# m.keep_in_front(NIL)
# folium.LayerControl().add_to(m)

In [ ]:
# # Display the map
# m